 ## Create the data

In [4]:
# import required modules
from mimesis.locales import Locale
from mimesis.keys import maybe
from mimesis.schema import Field, Schema
from mimesis.enums import TimestampFormat
from mimesis import Datetime

from random import Random
from uuid import UUID, uuid4

from bench_utils import generate_uuid4, get_gen_uuid4_unique_list

# Add table definitions

table_definition = {
    "person_amount": 1000,
    "product_amount": 1000,
    "order_amount":10000,
    "artist_amount": 500,
    "review_amount":2000
}

# Create table data

## person

person_id = generate_uuid4(table_definition['person_amount'], 10)

f = Field(locale=Locale.EN_GB, seed=10)

def person_generator() -> dict:
    first_name = f('first_name')
    last_name = f('last_name')
    return {
        "id":"person:"+f"⟨{str(next(person_id))}⟩",
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.9)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }

person_schema = Schema(
    schema=person_generator,
    iterations=table_definition['person_amount']
)
person_data = person_schema.create()

person_id_count = range(table_definition['person_amount']-1)

print("person data created")

## artist

artist_id = generate_uuid4(table_definition['artist_amount'], 20)

f = Field(locale=Locale.EN_GB, seed=20)

def artist_generator() -> dict:
    first_name = f("first_name")
    last_name = f("last_name")
    return {
        "id":"artist:"+f"⟨{str(next(artist_id))}⟩",
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.5)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }


artist_schema = Schema(
    schema=artist_generator,
    iterations=table_definition['artist_amount']
)

artist_data = artist_schema.create()

artist_id_count = range(table_definition['artist_amount']-1)

print("artist data created")

## product 

product_id = generate_uuid4(table_definition['product_amount'], 30)

f = Field(locale=Locale.EN_GB, seed=30)
dt = Datetime(seed=30)

rnd = Random()
rnd.seed(30)

def product_generator() -> dict:
    created_at = dt.timestamp(TimestampFormat.ISO_8601, start=2023, end=2023)+'Z'
    quantity = rnd.randint(1, 20)
    return {
        "id":"product:"+f"⟨{str(next(product_id))}⟩",
        "name":' '.join(f('words', quantity=2)),
        "description":' '.join(f('words', quantity=rnd.randint(8, 25))),
        "category":f('choice', items=["oil paint", "watercolor", "acrylic paint", "charcoal", "pencil", "ink", "pastel", "collage", "digital art", "mixed media"]),
        "price":f('price', minimum=500, maximum=25000),
        "currency":f('currency_symbol'),
        "discount":f('float_number', start=0.2, end=0.8, precision=1, key=maybe(None, probability=0.8)),
        "quantity":rnd.randint(1, 20), 
        "image_url":f('stock_image_url'),
        "artist":artist_data[f('choice', items=artist_id_count)]['id'],
        "creation_history": {
            "created_at":created_at,
            "quantity":quantity
        }
    }

product_schema = Schema(
    schema=product_generator,
    iterations=table_definition['product_amount']
)

product_data = product_schema.create()

product_id_count = range(table_definition['product_amount']-1)

print("product data created")

## order

order_id = generate_uuid4(table_definition['order_amount'], 40)

f = Field(locale=Locale.EN_GB, seed=40)
dt = Datetime(seed=40)

rnd = Random()
rnd.seed(40)

def order_generator() -> dict:
    person_number = f('choice', items=person_id_count)
    product_number = f('choice', items=product_id_count)
    shipping_address = person_data[person_number]['address']
    order_date = dt.timestamp(TimestampFormat.ISO_8601, start=2023, end=2023)+'Z'
    return {
        "id":f"order:"+f"⟨{str(next(order_id))}⟩",
        "in":person_data[person_number]['id'],
        "out":product_data[product_number]['id'],
        "product_name":product_data[product_number]['name'],
        "currency":product_data[product_number]['currency'],
        "discount":product_data[product_number]['discount'],
        "price":product_data[product_number]['price'],
        "quantity":rnd.randint(1, 3),
        "order_date":order_date,
        "shipping_address":shipping_address,
        "payment_method":f('choice', items=['credit card','debit card', 'PayPal']),
        "order_status":f('choice', items=['pending','processing', 'shipped', 'delivered'], key=maybe(None, probability=0.1))
    }

order_schema = Schema(
    schema=order_generator,
    iterations=table_definition['order_amount']
)

order_data = order_schema.create()

print("order data created")

## review

review_id = generate_uuid4(table_definition['review_amount'], 50)

f = Field(locale=Locale.EN_GB, seed=50)
dt = Datetime(seed=50)

rnd = Random()
rnd.seed(50)

def review_generator() -> dict:
    review_date = dt.timestamp(TimestampFormat.ISO_8601, start=2023, end=2023)+'Z'
    return {
        "id":"review:"+f"⟨{str(next(review_id))}⟩",
        "person":person_data[f('choice', items=person_id_count)]['id'],
        "product":product_data[f('choice', items=product_id_count)]['id'],
        "artist":artist_data[f('choice', items=artist_id_count)]['id'],
        "rating":f('choice', items=[1,2,3,4,5]),
        "review_text":' '.join(f('words', quantity=rnd.randint(8,50))),
        "review_date": review_date
    }

review_schema = Schema(
    schema=review_generator,
    iterations=table_definition['review_amount']
)

review_data = review_schema.create()

print("review data created")

person data created
artist data created
product data created
order data created
review data created


In [5]:
[product_data[i]['artist'] for i in range(5)]
# [83, 281, 174, 315, 361]

['artist:⟨db9fb768-148c-4388-9039-d1216cb6623f⟩',
 'artist:⟨37cc1e86-2114-495c-9f58-c240c8f2b42f⟩',
 'artist:⟨e857c13e-2f8d-41b0-86c9-423b775b26f3⟩',
 'artist:⟨184719af-792c-4505-9a96-814f6f8b1d3a⟩',
 'artist:⟨fa2f247f-e3ca-487b-a493-a3c2d13e66bf⟩']

In [ ]:
[UUID('db9fb768-148c-4388-9039-d1216cb6623f'),
 UUID('37cc1e86-2114-495c-9f58-c240c8f2b42f'),
 UUID('e857c13e-2f8d-41b0-86c9-423b775b26f3'),
 UUID('184719af-792c-4505-9a96-814f6f8b1d3a'),
 UUID('fa2f247f-e3ca-487b-a493-a3c2d13e66bf')]

 ## Load the data

In [3]:
from surrealdb import SurrealDB
from uuid import uuid4

db = SurrealDB("ws://localhost:8000/test/test")

db.signin({
    "username": "root",
    "password": "root",
})


def insert_relate_statement(table_data:list[dict]) -> str:
    """
    Inserting data through relate statement
    """

    table_record_id = -1
    for record in table_data:
        table_record_id += 1
        db.query(
    f"RELATE {table_data[table_record_id]['in']} -> {table_data[table_record_id]['id']} -> {table_data[table_record_id]['out']} CONTENT {record};"
            )

In [4]:
%%time
db.query(f"INSERT INTO person {person_data}")

CPU times: user 3.29 s, sys: 63 ms, total: 3.35 s
Wall time: 3.41 s


[{'address': {'address_line_1': '1310 Blaney',
   'address_line_2': 'apt. 10',
   'city': 'Ballymena',
   'coordinates': [-84.279118, -146.269714],
   'country': 'England',
   'post_code': 'TG1C 7OP'},
  'email': 'holds1871@live.com',
  'first_name': 'Anthony',
  'id': 'person:⟨781ca038-936d-4c0f-bf62-c3214eb8d5e9⟩',
  'last_name': 'Reilly',
  'name': 'Anthony Reilly',
  'phone': '01500 265774'},
 {'address': {'address_line_1': '477 Glenlevan',
   'city': 'Banbridge',
   'coordinates': [46.585326, -122.522646],
   'country': 'Scotland',
   'post_code': 'OS7K 4HW'},
  'email': 'dose1974@duck.com',
  'first_name': 'Laila',
  'id': 'person:⟨b7f70c53-685c-4bad-bdbc-5745539878d8⟩',
  'last_name': 'Sears',
  'name': 'Laila Sears',
  'phone': '0112 713 3320'},
 {'address': {'address_line_1': '866 Derryview',
   'city': 'Luton',
   'coordinates': [55.283089, 82.703443],
   'country': 'Scotland',
   'post_code': 'EY3C 0WP'},
  'email': 'cargo1950@protonmail.com',
  'first_name': 'Vivien',
  'id

In [5]:
%%time
db.query(f"INSERT INTO product {product_data}")

CPU times: user 1.14 s, sys: 22.8 ms, total: 1.16 s
Wall time: 1.22 s


[{'artist': 'artist:⟨86f337ce-8173-4bd1-812d-dc815b2fb415⟩',
  'category': 'pastel',
  'creation_history': {'created_at': '2023-01-24T08:15:14.146316',
   'quantity': 11},
  'currency': '£',
  'description': 'dr curve recognition please temple becomes hiv alliance crm leading alabama continental past marshall crystal mystery sh achieve served principle',
  'id': 'product:⟨ebe17946-dd08-48f8-86a5-7350d2854a3f⟩',
  'image_url': 'https://source.unsplash.com/1920x1080?',
  'name': 'page aims',
  'price': 4840.08,
  'quantity': 16},
 {'artist': 'artist:⟨7c1e4421-60f1-4306-989f-5d403b9da682⟩',
  'category': 'watercolor',
  'creation_history': {'created_at': '2023-11-24T17:05:37.442417',
   'quantity': 4},
  'currency': '£',
  'description': 'vintage famous bestsellers bills square nokia sessions rather mainly wedding garden measurement small',
  'id': 'product:⟨3a669d89-cf2d-4405-89bd-584aa1c9a1dd⟩',
  'image_url': 'https://source.unsplash.com/1920x1080?',
  'name': 'fear choice',
  'price':

In [6]:
%%time
insert_relate_statement(order_data)

CPU times: user 11.4 s, sys: 458 ms, total: 11.9 s
Wall time: 17.3 s


In [7]:
%%time
db.query(f"INSERT INTO artist {artist_data}")

CPU times: user 1.64 s, sys: 28.1 ms, total: 1.67 s
Wall time: 1.72 s


[{'address': {'address_line_1': '245 Kitcheners',
   'city': 'Harlow',
   'coordinates': [4.3413, -97.786325],
   'country': 'Northern Ireland',
   'post_code': 'JL9X 8VU'},
  'company_name': 'National Grid (NG.)',
  'email': 'catholic2093@protonmail.com',
  'first_name': 'Leonia',
  'id': 'artist:⟨85b837f7-cbba-49c8-854a-000d229b1481⟩',
  'last_name': 'Dillon',
  'name': 'Leonia Dillon',
  'phone': '0141 389 8590'},
 {'address': {'address_line_1': '415 Drumanee',
   'city': 'Barry',
   'coordinates': [67.07081, 169.357399],
   'country': 'Scotland',
   'post_code': 'KC3B 5NT'},
  'company_name': 'Restaurant Gp (RTN)',
  'email': 'answers1955@outlook.com',
  'first_name': 'Wilburn',
  'id': 'artist:⟨7ed3d6f5-500d-470e-949e-b6b1865f45a9⟩',
  'last_name': 'Christensen',
  'name': 'Wilburn Christensen',
  'phone': '0800 285089'},
 {'address': {'address_line_1': '115 Fennel',
   'city': 'Knighton',
   'coordinates': [-58.062479, 172.575897],
   'country': 'Scotland',
   'post_code': 'GU1Y 

In [8]:
%%time
db.query(f"INSERT INTO review {review_data}")

CPU times: user 1 s, sys: 20.3 ms, total: 1.02 s
Wall time: 1.09 s


[{'artist': 'artist:⟨1901fc91-fda1-4981-96dc-b0672e57f72d⟩',
  'id': 'review:⟨b55149a3-33a3-4a45-9f63-4d516c362402⟩',
  'person': 'person:⟨b9607687-76e7-4530-a2ac-7f54979a65d0⟩',
  'product': 'product:⟨02371b11-79d5-4a80-8130-b747a5a05483⟩',
  'rating': 4,
  'review_date': '2023-08-07T10:13:12.173875',
  'review_text': 'nurse pt idea chain axis complete enterprises atlantic earrings ny lowest seconds bringing real uniform toward minister stroke submission being structure increasingly sized happen nt when driven prayer billion classes principles break'},
 {'artist': 'artist:⟨00d6a569-9d84-453a-8b43-3b26474496f0⟩',
  'id': 'review:⟨a1f77691-18ea-4cc3-b16e-63d6ee918661⟩',
  'person': 'person:⟨a8d4d396-e443-4e95-8197-77e6fa803f47⟩',
  'product': 'product:⟨59ccfec6-5754-43e9-958d-8fdd44fe32f3⟩',
  'rating': 5,
  'review_date': '2023-01-05T16:49:52.621951',
  'review_text': 'maybe articles supposed hobbies again pound cnet abstract account undergraduate religious management console gratis su

 ## Run the queries

In [ ]:
from random import randint
# getting just an array of ids to use for loops in some queries
person_ids = db.query(f"SELECT VALUE id FROM person")
product_ids = db.query(f"SELECT VALUE id FROM product")
order_ids = db.query(f"SELECT VALUE id FROM order")
artist_ids = db.query(f"SELECT VALUE id FROM artist")
review_ids = db.query(f"SELECT VALUE id FROM review")

 ### Q1: lookup vs record links

In [9]:
%%timeit
db.query(""" 
SELECT
    id,
    rating,
    review_text,
	artist.name,
	artist.email,
	artist.phone,
	person.name,
	person.email,
	person.phone,
	product.name,
	product.category,
	product.price
FROM review;
""")

105 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


 ### Q2: lookup vs graph (and link)

In [ ]:
# %%timeit
db.query("""
SELECT
	price,
	order_date,
	product_name,
	->product.category,
	->product.description,
	->product.image_url,
	<-person.name,
	<-person.email,
	<-person.phone,
	->product.artist.name,
	->product.artist.email,
	->product.artist.phone
FROM order;
""")

 ### Q2: lookup vs graph - using in/out instead of arrow


In [ ]:
# %%timeit
db.query(""" 
SELECT
	price,
	order_date,
	product_name,
	out.category,
	out.description,
	out.image_url,
	in.person.name,
	in.person.email,
	in.person.phone,
	out.artist.name,
	out.artist.email,
	out.artist.phone
FROM order;
""")

 ### Q3: Name and email for all customers in England

In [ ]:
%%time
db.query(""" 
DEFINE INDEX person_country ON TABLE person COLUMNS address.country;
""")

In [ ]:
%%timeit
db.query(""" 
SELECT name, email 
FROM person 
WHERE address.country = "England";	
""")

 ### Q4: Count the number of confirmed orders in Q1 by artists in England

In [ ]:
%%time
db.query(""" 
DEFINE INDEX order_count ON TABLE order COLUMNS order_status, order_date, address.country;
""")

In [ ]:
%%timeit
db.query(""" 
SELECT count() FROM order
WHERE (order_status != "pending"
AND order_status = null)
AND time::month(<datetime>order_date) <=3
AND ->product.artist.address.country ?= "England"
GROUP ALL;
""")


In [ ]:
db.query(""" 
SELECT count() FROM order
WHERE order_status IN ["delivered", "processing", "shipped"]
AND time::month(<datetime>order_date) <4 
AND ->product.artist.address.country ?= "England"
GROUP ALL;
""")

 ### Q5: Delete a specific review

In [ ]:
%%time
db.query(f""" 
DELETE {review_ids[0]};
""")


 ### Q6: Delete reviews from a particular category

In [ ]:
%%time
db.query(""" 
DELETE review
WHERE product.category = "charcoal";
""")

 ### Q7: Update a customer address

In [ ]:
%%timeit
db.query(f"UPDATE {person_ids[randint(0, person_id_count)]}"+"""
SET address = {
	'address_line_1': '497 Ballycander',
	'address_line_2': None,
	'city': 'Bromyard',
	'country': 'Wales',
	'post_code': 'ZX8N 4VJ',
	'coordinates': [68.772592, -35.491877]
	}
RETURN NONE;
""")

 ### Q8: Update discounts for products

In [ ]:
%%time
db.query(""" 
UPDATE product
SET discount = 0.2
WHERE price < 1000
RETURN NONE;
""")


 ### Q9: Transaction - order from a new customer

In [ ]:
# TODO do this with record links to compare against mongo
%%timeit
random_person_id = person_ids[randint(0, person_id_count)]
random_product_id = product_ids[randint(0, product_id_count)]

db.query(""" 
# Transaction - order from a new customer
BEGIN TRANSACTION;
-- insert into the person table
CREATE person CONTENT {
	"""+f"'id': {random_person_id},"+"""
	'first_name': 'Karyl',
	'last_name': 'Langley',
	'name': 'Karyl Langley',
	'company_name': None,
	'email': 'dee1961@gmail.com',
	'phone': '+44 47 3516 5895',
	'address': {
		'address_line_1': '510 Henalta',
		'address_line_2': None,
		'city': 'Lyme Regis',
		'country': 'Northern Ireland',
		'post_code': 'TO6Q 8CM',
		'coordinates': [-34.345071, 118.564172]
		}
	};

-- relate into the order table"""+
f"RELATE {random_person_id} -> order:uuid() -> {random_product_id}"+"""
CONTENT {
        "currency": "£",
        "discount": ->product.discount,
        "order_date": time::now(),
        "order_status": "pending",
        "payment_method": "PayPal",
        "price": ->product.price,
        "product_name": ->product.name,
        "quantity": 1,
        "shipping_address": <-person.address
	};

-- update the product table to reduce the quantity"""+
f"""
UPDATE {random_product_id} SET quantity -= 1 RETURN NONE;
COMMIT TRANSACTION;
""")


 ### Q10: "Transaction"* - New Artist creates their first product

In [ ]:
%%timeit
# Transaction - New Artist creates their first product

new_artist_id = str(uuid4())
new_product_id = str(uuid4())

db.query(""" 
BEGIN TRANSACTION;
-- insert into the artist table
CREATE artist CONTENT {"""+
        f"'id': 'artist:⟨{new_artist_id}⟩',"+"""
        'first_name': 'Anderson',
        'last_name': 'West',
        'name': 'Anderson West',
        'company_name': 'Atkins(ws) (ATK)',
        'email': 'six1933@gmail.com',
        'phone': '056 5881 1126',
        'address': {
                'address_line_1': '639 Connaugh',
                'address_line_2': None,
                'city': 'Ripon',
                'country': 'Scotland',
                'post_code': 'CG3U 4TH',
                'coordinates': [4.273648, -112.907273]
                }
        };

-- insert into the product table
CREATE product CONTENT {"""+
        f"'id': 'product:⟨{new_product_id}⟩',"+"""
        'name': 'managed edt allocated pda',
        'description': 'counseling dildo greek pan works interest xhtml wrong dennis available cl specific next tower webcam peace magic',
        'category': 'watercolor',
        'price': 15735.96,
        'currency': '£',
        'discount': None,
        'quantity': 1,
        'image_url': 'https://source.unsplash.com/1920x1080?',"""
        f"'artist': 'artist:⟨{new_artist_id}⟩',"+"""
        "creation_history": {
                "quantity": 1,
                "created_at": time::now()
                }       
        };
COMMIT TRANSACTION;
""")

In [10]:
dt.timestamp(TimestampFormat.ISO_8601, start=2023, end=2023)+'Z'

'2023-07-24T09:03:50.029655Z'